In [1]:
import numpy as np
import pandas as pd
from keras.layers import Dense
from keras.layers import Dropout
from keras.layers import LSTM
from keras.models import Sequential
from sklearn.preprocessing import MinMaxScaler
import matplotlib.pyplot as plt
%matplotlib inline

Using TensorFlow backend.
/anaconda3/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:526: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/anaconda3/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:527: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/anaconda3/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:528: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/anaconda3/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:529: FutureWarn

In [2]:
dataset = pd.read_csv('fyp.csv') 

In [3]:
dataset.shape

(4935, 7)

In [ ]:
prices_dataset_train = dataset.iloc[:3948,:]
prices_dataset_test = dataset.iloc[3949:,:]

In [ ]:
trainingset = dataset.iloc[:3948,5:6].values
testset = dataset.iloc[3949:,5:6].values

In [ ]:
trainingset

In [ ]:
type(trainingset)

In [ ]:
trainingset.shape

In [ ]:
# NORMALIZATION

In [ ]:
min_max_scaler = MinMaxScaler(feature_range=(0,1))
scaled_trainingset = min_max_scaler.fit_transform(trainingset)

In [ ]:
X_train = []
y_train = []

In [ ]:
for i in range(20,3948):
    X_train.append(scaled_trainingset[i-20:i,0])
    y_train.append(scaled_trainingset[i,0])

In [ ]:
X_train = np.array(X_train)
y_train = np.array(y_train)

In [ ]:
X_train = np.reshape(X_train, (X_train.shape[0],X_train.shape[1],1))

In [ ]:
# LSTM MODEL

In [ ]:
# LSTM architecture

In [ ]:
model = Sequential()
model.add(LSTM(units=100, return_sequences=True, input_shape=(X_train.shape[1],1)))
model.add(Dropout(0.5))
model.add(LSTM(units=50, return_sequences=True))
model.add(Dropout(0.3))
# model.add(LSTM(units=50, return_sequences=True))
# model.add(Dropout(0.3))
model.add(LSTM(units=50))
model.add(Dropout(0.3))
model.add(Dense(units=1))

In [ ]:
# Optimizer
model.compile(optimizer='adam', loss='mean_squared_error')
model.fit(X_train, y_train, epochs=50, batch_size=32)

In [ ]:
dataset_total = dataset.iloc[:,5:6]

In [ ]:
inputs = dataset_total[len(dataset_total)-len(prices_dataset_test)-20:].values
inputs = inputs.reshape(-1,1)

In [ ]:
inputs = min_max_scaler.transform(inputs)

In [ ]:
X_test = []

In [ ]:
for i in range(20,len(prices_dataset_test)+20):
    X_test.append(inputs[i-20:i,0])

In [ ]:
X_test = np.array(X_test)
X_test = np.reshape(X_test, (X_test.shape[0],X_test.shape[1],1))

In [ ]:
predictions = model.predict(X_test)

In [ ]:
predictions = min_max_scaler.inverse_transform(predictions)

In [ ]:
# Plotting
plt.plot(testset, color='blue', label='Stock Price')
plt.plot(predictions, color='green', label='LSTM Predictions')
plt.title('Stock Price Prediction with RNN/LSTM')
plt.xlabel('Time')
plt.ylabel('Price')
plt.legend()
plt.show()